In [5]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import skimage.io
from scipy.ndimage import median_filter
from scipy import signal
from skimage import exposure
import multipagetiff as mtif
# import cloudComPy as cc
# from n2v.models import N2VConfig, N2V
# from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
# from csbdeep.utils import plot_history
# from n2v.utils.n2v_utils import manipulate_val_data, autocorrelation
# from google.colab.patches import cv2_imshow
# from google.colab import drive
import pyvista as pv
from pyntcloud import PyntCloud

https://cloudconvert.com/tif-to-jpg

* https://ieeexplore.ieee.org/document/8954066
* https://octave.org/
* https://github.com/cloudcompare/cloudcompare / https://www.danielgm.net/cc/
* https://scholar.google.com/scholar?oi=bibs&hl=en&cites=14009452987874973669&authuser=1
* https://pointclouds.org/documentation/classpcl_1_1_morphology.html
* https://scholar.google.com/scholar?hl=en&as_sdt=2005&sciodt=0%2C5&cites=9800399061227290708&scipsc=1&q=pointclouds&authuser=1

checked
--- 
* https://arxiv.org/pdf/2206.03977v1.pdf
* https://scikit-learn.org/stable/modules/generated/sklearn.manifold.SpectralEmbedding.html
* https://projecteuclid.org/journals/duke-mathematical-journal/volume-8/issue-2/Riemannian-manifolds-with-positive-mean-curvature/10.1215/S0012-7094-41-00832-3.short
* https://inside.mines.edu/~whereman/papers/delaPorte-Herbst-Hereman-vanderWalt-PRASA-2008.pdf

In [2]:
# from n2v.models import N2VConfig, N2V
# from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
# import numpy as np
# # Create a data generator
# datagen = N2V_DataGenerator()

# # Load your data
# stack = mtif.read_stack("testimages/2022.12.21_APS018_Infection_PA14-GacA_40X_40%_EM-4_D2_3_7047.tif", units='um')
# stack= np.array(stack) 
# stack = stack[..., None]
# stack.shape

In [3]:
# config = N2VConfig(stack, unet_kern_size=3, 
#                    train_steps_per_epoch=10, train_epochs=30, train_loss='mse', batch_norm=True, 
#                    train_batch_size=128, n2v_perc_pix=0.198, n2v_patch_shape=(64, 64), 
#                    unet_n_first = 96,
#                    unet_residual = True,
#                    n2v_manipulator='normal_withoutCP', n2v_neighborhood_radius=2,
#                    structN2Vmask = [[0,1,1,1,1,1,0]]) ## mask should be wide enough to cover most of the noise autocorrelation


In [4]:
# # the base directory in which our model will live
# basedir = 'models'
# # We are now creating our network model.
# model = N2V(config, 'n2v_2D', basedir=basedir)
# model.prepare_for_training(metrics=())

In [5]:
# out = model.train(stack, stack)

In [49]:
import open3d as o3d
import numpy as np

# Load a stack of grayscale images
images = mtif.read_stack("testimages/2022.12.21_APS018_Infection_PA14-GacA_40X_40%_EM-4_D2_3_7047.tif", units='um')

# Create an empty point cloud
x_0 = o3d.geometry.PointCloud()

# Add points to the point cloud for each pixel in the images
for z, image in enumerate(images):
    height, width = np.asarray(image).shape
    for y in range(height):
        for x in range(width):
            # Create a point for the current pixel
            point = [x, y, z]
            # Set the intensity of the point to the pixel value
            intensity = np.asarray(image)[y, x] / 255.0
            # Add the point to the point cloud
            x_0.points.append(point)
            x_0.colors.append([intensity, intensity, intensity])

# Scale the point cloud to improve visualization
x_0.scale(0.1, center=x_0.get_center())

# # Visualize the point cloud
# o3d.visualization.draw_geometries([x_0])

# Convert the Open3D point cloud to a PyVista point cloud
points = np.asarray(x_0.points)
colors = np.asarray(x_0.colors)
pyvista_point_cloud = pv.PolyData(points)
pyvista_point_cloud['colors'] = colors

# Visualize the point cloud
pyvista_point_cloud.plot(render_points_as_spheres=True)

Widget(value="<iframe src='http://localhost:52238/index.html?ui=P_0x27927b56310_6&reconnect=auto' style='width…

In [54]:
# import numpy as np
# import open3d as o3d

# # Load the 3D image stack
# images = mtif.read_stack("testimages/2022.12.21_APS018_Infection_PA14-GacA_40X_40%_EM-4_D2_3_7047.tif", units='um')

# # # Extract pixel coordinates and map them to 3D space
# # coords = []
# # for i, image in enumerate(images):
# #     coords_i = np.column_stack(np.where(image > 0))
# #     coords_i = np.hstack((coords_i, np.full((coords_i.shape[0], 1), i)))  # Mapping to Z-coordinate
# #     coords.append(coords_i)

# # # Combine the pixel coordinates from all images
# # all_coords = np.concatenate(coords)


# # # Create a point cloud
# # x_0 = o3d.geometry.PointCloud()
# # x_0.points = o3d.utility.Vector3dVector(all_coords)

# # # Visualize the point cloud
# # o3d.visualization.draw_geometries([point_cloud])


In [20]:
images = np.array(images)

In [21]:
for img in images:
  cv2.imshow('image', img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [22]:
np_images = np.stack(images, axis=2)
np_equalised_images = exposure.equalize_adapthist(np_images, clip_limit=0.03)
equalised_images = np.split(np_equalised_images, np_equalised_images.shape[2], axis=2)
equalised_images = [np.squeeze(img) for img in equalised_images]

# try applying at different stages 

In [48]:
np.asarray(equalised_images).shape


(14, 512, 512)

In [52]:
# Create an empty point cloud
x_1 = o3d.geometry.PointCloud()

# Add points to the point cloud for each pixel in the images
for z, image in enumerate(equalised_images):
    height, width = np.asarray(image).shape
    for y in range(height):
        for x in range(width):
            # Create a point for the current pixel
            point = [x, y, z]
            # Set the intensity of the point to the pixel value
            intensity = np.asarray(image)[y, x] / 255.0
            # Add the point to the point cloud
            x_1.points.append(point)
            x_1.colors.append([intensity, intensity, intensity])

# Scale the point cloud to improve visualization
x_1.scale(0.1, center=x_0.get_center())

# # Visualize the point cloud
# o3d.visualization.draw_geometries([x_0])

# Convert the Open3D point cloud to a PyVista point cloud
points = np.asarray(x_1.points)
colors = np.asarray(x_1.colors)
pyvista_point_cloud = pv.PolyData(points)
pyvista_point_cloud['colors'] = colors

# Visualize the point cloud
pyvista_point_cloud.plot(render_points_as_spheres=True)

Widget(value="<iframe src='http://localhost:52238/index.html?ui=P_0x27927adb190_7&reconnect=auto' style='width…

In [10]:
# for eqimg in equalised_images:
#     cv2.imshow('image', eqimg)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [29]:
eq_images = np.stack(equalised_images, axis=2)
med_blur_images_numpyarray = signal.medfilt(eq_images, kernel_size=(3,3,3))
med_blur_images = np.split(med_blur_images_numpyarray, med_blur_images_numpyarray.shape[2], axis=2)
med_blur_images = [np.squeeze(img) for img in med_blur_images]

In [53]:
# Create an empty point cloud
x_2 = o3d.geometry.PointCloud()

# Add points to the point cloud for each pixel in the images
for z, image in enumerate(med_blur_images):
    height, width = np.asarray(image).shape
    for y in range(height):
        for x in range(width):
            # Create a point for the current pixel
            point = [x, y, z]
            # Set the intensity of the point to the pixel value
            intensity = np.asarray(image)[y, x] / 255.0
            # Add the point to the point cloud
            x_2.points.append(point)
            x_2.colors.append([intensity, intensity, intensity])

# Scale the point cloud to improve visualization
x_2.scale(0.1, center=x_0.get_center())

# # Visualize the point cloud
# o3d.visualization.draw_geometries([x_0])

# Convert the Open3D point cloud to a PyVista point cloud
points = np.asarray(x_2.points)
colors = np.asarray(x_2.colors)
pyvista_point_cloud = pv.PolyData(points)
pyvista_point_cloud['colors'] = colors

# Visualize the point cloud
pyvista_point_cloud.plot(render_points_as_spheres=True)

Widget(value="<iframe src='http://localhost:52238/index.html?ui=P_0x27927bf93a0_8&reconnect=auto' style='width…

In [12]:
for med_img in med_blur_images:
    cv2.imshow('image', med_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [33]:
def convert_float64_to_uint8(float64_images):
    uint8_images = []
    for float64_image in float64_images:
        # Normalize pixel values to the range [0, 255]
        norm_image = ((float64_image - float64_image.min()) / (float64_image.max() - float64_image.min())) * 255
        # Convert data type to uint8
        uint8_image = norm_image.astype(np.uint8)
        uint8_images.append(uint8_image)
    return uint8_images

uint8_med_blur = convert_float64_to_uint8(med_blur_images)

In [34]:
for med_img in uint8_med_blur:
    cv2.imshow('image', med_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

point cloud after median filtering

After 3d equalsisation, top and bottom images are still darker than the central images in the stack
so 2d otsu's will set lower thresholds for top and bottom images than for central images


In [35]:
# binary_images = []
# for med_img in uint8_med_blur:
#     _, binary_image = cv2.threshold(med_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     # binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

#     # _, binary_image = cv2.threshold(eqimg, 0, 255, cv2.THRESH_BINARY)
#     binary_images.append(binary_image)

binary_images = []
for med_img in uint8_med_blur:
    _, binary_image = cv2.threshold(med_img, 135, 255, cv2.THRESH_BINARY)
    binary_images.append(binary_image)

In [54]:
# Create an empty point cloud
x_3 = o3d.geometry.PointCloud()

# Add points to the point cloud for each pixel in the images
for z, image in enumerate(binary_images):
    height, width = np.asarray(image).shape
    for y in range(height):
        for x in range(width):
            # Create a point for the current pixel
            point = [x, y, z]
            # Set the intensity of the point to the pixel value
            intensity = np.asarray(image)[y, x] / 255.0
            # Add the point to the point cloud
            x_3.points.append(point)
            x_3.colors.append([intensity, intensity, intensity])

# Scale the point cloud to improve visualization
x_3.scale(0.1, center=x_0.get_center())

# # Visualize the point cloud
# o3d.visualization.draw_geometries([x_0])

# Convert the Open3D point cloud to a PyVista point cloud
points = np.asarray(x_3.points)
colors = np.asarray(x_3.colors)
pyvista_point_cloud = pv.PolyData(points)
pyvista_point_cloud['colors'] = colors

# Visualize the point cloud
pyvista_point_cloud.plot(render_points_as_spheres=True)

Widget(value="<iframe src='http://localhost:52238/index.html?ui=P_0x2792a81f100_9&reconnect=auto' style='width…

In [16]:
for bin_img in binary_images:
    cv2.imshow('binary image', bin_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [17]:
kernel_size = 5
kernel = cv2.getStructuringElement(cv2.MORPH_OPEN, (kernel_size, kernel_size))

# Apply morphological closing operation to each binary image
closed_images = []
for binary_image in binary_images:
    closed_img = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel, iterations=0)
    closed_images.append(closed_img)

In [18]:
for cls_img in closed_images:
    # plt.imshow(bin_img, cmap='gray')
    # plt.show()
    cv2.imshow('binary image', cls_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [72]:
# Extract pixel coordinates and map them to 3D space
coords = []
for i, image in enumerate(closed_images):
    coords_i = np.column_stack(np.where(image > 0))
    coords_i = np.hstack((coords_i, np.full((coords_i.shape[0], 1), -i)))  # Mapping to Z-coordinate
    coords.append(coords_i)

# Combine the pixel coordinates from all images
all_coords = np.concatenate(coords)

# Scale and normalize the coordinates
scale_factor_z = 3  # Adjust the scale factor for the z-axis as needed
all_coords[:, 2] = all_coords[:, 2] * scale_factor_z

# Calculate the center of the point cloud
center = np.mean(all_coords, axis=0)

# Translate the coordinates to center the point cloud
all_coords[:, 0] = all_coords[:, 0] - center[0]
all_coords[:, 1] = all_coords[:, 1] - center[1]
all_coords[:, 2] = all_coords[:, 2] - center[2]

# Create a point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(all_coords[:, [1, 0, 2]])  # Swap X and Y axes for Open3D

In [73]:
# Visualize the point cloud
downpcd = point_cloud.voxel_down_sample(voxel_size=0.0001)

o3d.visualization.draw_geometries([point_cloud],
                                  zoom=1,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[0, 0, 0],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True,)


In [74]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
###################################################################
This Source Code Form is subject to the terms of the Mozilla Public
License, v. 2.0. If a copy of the MPL was not distributed with this
file, You can obtain one at https://mozilla.org/MPL/2.0/.

copyright (c) 2020, Peter Szutor

@author: Peter Szutor, Hungary, szppaks@gmail.com
Created on Wed Feb 26 17:23:24 2020
###################################################################



Octree-based lossy point-cloud compression with open3d and numpy
Average compressing rate (depends on octreee depth setting parameter): 0.012 - 0.1


Input formats: You can get a list of supported formats from : http://www.open3d.org/docs/release/tutorial/Basic/file_io.html#point-cloud
               (xyz,pts,ply,pcd)

Usage:

Dependencies: Open3D, Numpy  (You can install theese modules:  pip install open3d, pip install numpy)

Compress a point cloud:

octreezip(<filename>,<depth>) -> <result>
<filename>: (str) Point Cloud file name. Saved file name: [filename without ext]_ocz.npz  (Yes, it's a numpy array file)
<depth>   : (str) Octree depth. You can try 11-16 for best result. Bigger depht results higher precision and bigger compressed file size.
<result>  : (str) If the compressing was success you get: "Compressed into:[comp.file name] | Storing resolution:0.003445". Storing resolution means the precision.
                  The PC file is missing or bad: "PC is empty, bad, or missing"
                  Other error: "Error: [error message]"


Uncompressing:
octreeunzip(<filename>) -> <result>
<filename>: (str) Zipped Point Cloud file name (npz). Saved file name: [filename].xyz  (standard XYZ text file)
<result>  : (str) If the compressing was success you get: "Saved: [filename].xyz"
                  Other error: "Error: [error message]"
"""
import numpy as np
import os
import open3d as o3d

def d1halfing_fast(pmin,pmax,pdepht):
    return np.linspace(pmin,pmax,2**int(pdepht)+1)

def octreecodes(ppoints,pdepht):
    minx=np.amin(ppoints[:,0])
    maxx=np.amax(ppoints[:,0])
    miny=np.amin(ppoints[:,1])
    maxy=np.amax(ppoints[:,1])
    minz=np.amin(ppoints[:,2])
    maxz=np.amax(ppoints[:,2])
    xletra=d1halfing_fast(minx,maxx,pdepht)
    yletra=d1halfing_fast(miny,maxy,pdepht)
    zletra=d1halfing_fast(minz,maxz,pdepht)
    otcodex=np.searchsorted(xletra,ppoints[:,0],side='right')-1
    otcodey=np.searchsorted(yletra,ppoints[:,1],side='right')-1
    otcodez=np.searchsorted(zletra,ppoints[:,2],side='right')-1
    ki=otcodex*(2**(pdepht*2))+otcodey*(2**pdepht)+otcodez
    return (ki,minx,maxx,miny,maxy,minz,maxz)

def octreezip(pfilename,pdepht):
    try:
        pcd = o3d.io.read_point_cloud(pfilename,format='auto')
        ppoints=np.asarray(pcd.points)
        if len(ppoints)>0:
            occ=octreecodes(ppoints,pdepht)
            occsorted=np.sort(occ[0])
            prec=np.amax(np.asarray([occ[2]-occ[1],occ[4]-occ[3],occ[6]-occ[5]])/(2**pdepht))
            paramarr=np.asarray([pdepht,occ[1],occ[2],occ[3],occ[4],occ[5],occ[6]]) #depth and boundary
            np.savez_compressed(os.path.splitext(pfilename)[0]+'_ocz',points=occsorted,params=paramarr)
            retmessage='Compressed into:'+str(os.path.splitext(pfilename)[0])+'.ocz | Storing resolution:'+str(prec)
        else:
            retmessage='PC is empty, bad, or missing'
    except Exception as e:
        retmessage='Error:'+str(e)
    return retmessage

def octreeunzip(pfilename):
    try:
        pc=np.load(pfilename)
        pcpoints=pc['points']
        pcparams=pc['params']
        pdepht=(pcparams[0])
        minx=(pcparams[1])
        maxx=(pcparams[2])
        miny=(pcparams[3])
        maxy=(pcparams[4])
        minz=(pcparams[5])
        maxz=(pcparams[6])
        xletra=d1halfing_fast(minx,maxx,pdepht)
        yletra=d1halfing_fast(miny,maxy,pdepht)
        zletra=d1halfing_fast(minz,maxz,pdepht)
        occodex=(pcpoints/(2**(pdepht*2))).astype(int)
        occodey=((pcpoints-occodex*(2**(pdepht*2)))/(2**pdepht)).astype(int)
        occodez=(pcpoints-occodex*(2**(pdepht*2))-occodey*(2**pdepht)).astype(int)
        koorx=xletra[occodex]
        koory=yletra[occodey]
        koorz=zletra[occodez]
        points=np.array([koorx,koory,koorz]).T
        np.savetxt(os.path.splitext(pfilename)[0]+'.xyz',points,fmt='%.4f')
        retmessage='Saved:'+os.path.splitext(pfilename)[0]+'.xyz'
    except Exception as e:
        retmessage='Error:'+str(e)
    return retmessage

In [75]:
# Save the point cloud
o3d.io.write_point_cloud("point_cloud.ply", point_cloud)
octreezip('point_cloud.ply', 8)

'Compressed into:point_cloud.ocz | Storing resolution:1.9921875'

In [47]:
octreeunzip('point_cloud_ocz.npz')

'Saved:point_cloud_ocz.xyz'

In [48]:
# Uncompress the point cloud


# Read the uncompressed point cloud from file
comp1pcd = o3d.io.read_point_cloud('point_cloud_ocz.xyz')

# type(compressed1pcd)

In [49]:
# compressed1pcd = octreeunzip('point_cloud_ocz.npz')

# Visualize the compressed point cloud
o3d.visualization.draw_geometries([comp1pcd],
                                  zoom=1,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[0, 0, 0],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)


In [24]:
# # Create a mesh from the point cloud using alpha shape algorithm
# alpha = 5
# print(f"alpha={alpha:.3f}")
# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, alpha)
# mesh.compute_vertex_normals()

In [25]:
# o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

- incorporate cylinders and knowledge of cylinders
- computer graphics and curvature
- 

In [26]:
# import networkx as nx
# from GraphRicciCurvature.OllivierRicci import OllivierRicci

# # First, you need to convert your 3D mesh into a NetworkX graph
# G = nx.Graph()

# # Add nodes to the graph
# for i, vertex in enumerate(mesh.vertices):
#     G.add_node(i, pos=vertex)

# # Add edges to the graph
# for triangle in mesh.triangles:
#     G.add_edge(triangle[0], triangle[1])
#     G.add_edge(triangle[1], triangle[2])
#     G.add_edge(triangle[2], triangle[0])

# # Compute Ricci curvature using the GraphRicciCurvature library
# orc = OllivierRicci(G, alpha=10, verbose="INFO")
# G_orc = orc.compute_ricci_curvature()


In [27]:
# from pydiffmap import diffusion_map as dm
# # from pydiffmap.kernel import VariableBandwidthKernel

# data = np.asarray(point_cloud.points)

# # Construct a Gaussian kernel object
# kernel_object = dm.kernel.Kernel(kernel_type='gaussian', epsilon='bgh')

# # Create an instance of the DiffusionMap class
# dmap = dm.DiffusionMap(kernel_object=kernel_object, n_evecs=3)

# # Compute the diffusion map
# dmap_data = dmap.fit_transform(data)

# from pydiffmap import visualization as vis

# vis.data_plot(dmap)

- compute a diffusion map reducing from 3d point cloud to 2d
- fit a line through the diffusion map (PCA?)\
- https://en.wikipedia.org/wiki/Curvature
- Calculate curvature by seeing how much the generated line deviates from a straight line
- 
